In [0]:
import copy

In [0]:
def replace_literal(D, literal, value):
    num_clauses = len(D)
    D_copy = copy.copy(D)
    for i in range(num_clauses):
        D_copy[i] = [value if l == literal else l for l in D_copy[i]]
        D_copy[i] = ['-{}'.format(value) if l == -literal else l for l in D_copy[i]]
        
    return D_copy

In [3]:
D = [[1,2,-1],[-1, -2]]
     
replace_literal(D, 1, 'T')

[['T', 2, '-T'], ['-T', -2]]

In [0]:
def parse_dimacs(dimacs_formula):
    """
    Funkcija parsira formulu zapisanu u dimacs formatu
    i vraca listu listi, odnosno listu klauza, pri cemu
    su literali reprezentovani brojevima, a njihove negacije
    negativnim brojevima.
    """
    lines = dimacs_formula.split('\n')
    
    D = []
    
    for line in lines:
        if len(line) == 0 or line[0] == 'c' or line[0] == 'p':
            continue;
            
        clause = []
        
        literals = [int(l) for l in line.split(' ')]
        
        for literal in literals:
            if literal != 0:
                clause.append(literal)
                
        D.append(clause)
        
    return D

In [0]:
def DPLL(dimacs_formula):
    D = parse_dimacs(dimacs_formula)
    print('START:')
    print(D)
    return _DPLL(D, {})

def _DPLL(D, valuation):
    num_clauses = len(D)
    
    if num_clauses == 0:
        return True, valuation
    
    # ---------------------------------------------------
    # {NEGATION SUPSTITION}
    # Menjamo ~F -> T, ~T -> F
    # ---------------------------------------------------
    # Menjamo ~T -> F i ~F -> T
    new_D = []
    has_changes = False
    # Idemo klauzu po klauzu
    for i in range(num_clauses):
        clause = D[i]
        m = len(clause)
        
        # Ako smo nasli praznu klauzu
        if m == 0:
            return False, None
        
        # Iteriramo kroz literale u i-toj klauzi
        for j in range(m):
            if clause[j] == '-T':
                clause[j] = 'F'
                has_changes = True
            
            elif clause[j] == '-F':
                clause[j] = 'T'
                has_changes = True
        
        new_D.append(clause)
        
    D = copy.copy(new_D)
    
    if has_changes:
        print('NEGATION SUBSTITUTION:')
        print(D)
        
        
    # Brisemo sve literale jedna F:
    new_D = []
    has_changes = False
    for clause in D:
        new_clause = []
        for l in clause:
            if l != 'F':
                new_clause.append(l)
            else:
                has_changes = True
        new_D.append(new_clause)
    
    if has_changes:
        D = copy.copy(new_D)
        print("DELETED F from formula")
        print(D)

    # Proveravamo da li se pojavila prazna klauza (ovo verovatno nije neophodno)
    for clause in D:
        if len(clause) == 0:
            return False, None
        
    # ---------------------------------------------------
    # {TAUTOLOGY}
    # Ako neka klauza Ci sadrzi T ili sadrzi literal i
    # njegovu negaciju, vracamo vrednost koju vraca
    # DPLL(D \ Ci).
    # ---------------------------------------------------
    new_D = []
    has_changes = False
       
    for clause in D:
        new_clause = []
        keep_clause = True
        
        for literal in clause:
            if literal == 'T':
                keep_clause = False
                has_changes = True
                break
                
            if literal != 'F':
                if -literal in clause:
                    # Postoji literal i njegova negacija u klauzi
                    keep_clause = False
                    has_changes = True
                    break
                else:
                    # Inace zadrzavamo literal
                    new_clause.append(literal)
            else:
                has_changes = True
                
        if keep_clause:
            new_D.append(new_clause)
            
    D = copy.copy(new_D)
    if has_changes:
        print('TAUTOLOGY:')
        print(D)
        return _DPLL(D, valuation)
    
    # ---------------------------------------------------
    # {UNIT PROPAGATION}
    # Ako je neka klauza jedinicna i jednaka nekom iskaznom
    # slovu p, onda vracamo DPLL(D[p -> T])
    # Ako je neka klauza jedinicna i jednaka nekom iskaznom
    # slovu ~p, onda vracamo DPLL(D[p -> F])
    # ---------------------------------------------------
    new_D = []
    has_changes = False
    for clause in D:
        new_clause = []
        keep_clause = True
        m = len(clause)
        if m == 1:
            # Jedinicna klauza
            has_changes = True
            l = clause[0]
            if l > 0:
                valuation[l] = True
                new_D = replace_literal(D, abs(l), 'T')
            else:
                valuation[-l] = False
                new_D = replace_literal(D, abs(l), 'F')
            break
        if keep_clause:
            new_D.append(clause)
    D = copy.copy(new_D)
    if has_changes:
        print('UNIT PROPAGATION')
        print(D)
        return _DPLL(D, valuation)
    
    # ---------------------------------------------------
    # {PURE LITERAL}
    # Ako D sadrzi literal p, ali ne i ~p,
    # onda vracamo DPLL(D[p -> T]).
    # Ako D sadrzi literal ~p, ali ne i p,
    # onda vracamo DPLL(D[p -> F])
    # ---------------------------------------------------
    new_D = []
    has_changes = False
    all_literals = []
    for clause in D:
        all_literals += clause
        
    # Nadjemo sve jedinstvene literale
    distinct_literals = list(set([l for l in all_literals]))
    distinct_literals
    print(distinct_literals)
    for l in distinct_literals:
        # Proveravamo da li za l ne postoji njegova negacija
        if -l not in all_literals:
            has_changes = True
            if l > 0:
                valuation[l] = True
                new_D = replace_literal(D, abs(l), 'T')
            else:
                valuation[-l] = False
                new_D = replace_literal(D, abs(l), 'F')
            break
            
    if has_changes:
        print('PURE LITERAL:')
        D = copy.copy(new_D)
        print(D)
        return _DPLL(D, valuation)
    
    # ---------------------------------------------------
    # {SPLIT}
    # ---------------------------------------------------
    new_D = []
    # Ovo je vrlo vazan korak algoritma koji moze
    # drasticno uticati na efikasnost izavrsavanja algoritma,
    # jer odabir literala za korak SPLIT utice na dalji tok
    # pretrage.
    selected_literal = distinct_literals[0]
    valuation[selected_literal] = True
    new_D = replace_literal(D, selected_literal, 'T')
    print('SPLIT [{}->T]:'.format(selected_literal))
    print(new_D)
    res, valuation = _DPLL(new_D, valuation)
    
    if res:
        return True, valuation
    
    valuation[selected_literal] = False
    new_D = replace_literal(D, selected_literal, 'F')
    
    print('SPLIT [{}->F]:'.format(selected_literal))
    print(new_D)
    return _DPLL(new_D, valuation)

In [6]:
dimacs = 'p cnf 3 2\n1 2 3 0\n-1 -2 -3 0\n3 0'

#test_D = [[-3, 4],[-1, 2, -3],[3,1,-2],['-T', '-F']]
#_DPLL(D, {})

DPLL(dimacs)

START:
[[1, 2, 3], [-1, -2, -3], [3]]
UNIT PROPAGATION
[[1, 2, 'T'], [-1, -2, '-T'], ['T']]
NEGATION SUBSTITUTION:
[[1, 2, 'T'], [-1, -2, 'F'], ['T']]
DELETED F from formula
[[1, 2, 'T'], [-1, -2], ['T']]
TAUTOLOGY:
[[-1, -2]]
[-1, -2]
PURE LITERAL:
[['-F', -2]]
NEGATION SUBSTITUTION:
[['T', -2]]
TAUTOLOGY:
[]


(True, {1: False, 3: True})

In [7]:
_DPLL([['F', 1], ['T', 2]], {})

DELETED F from formula
[[1], ['T', 2]]
TAUTOLOGY:
[[1]]
UNIT PROPAGATION
[['T']]
TAUTOLOGY:
[]


(True, {1: True})